# How to add tools to chatbots

:::info Prerequisites

This guide assumes familiarity with the following concepts:

- [Chatbots](/docs/concepts/#messages)
- [Agents](/docs/tutorials/agents)
- [Chat history](/docs/concepts/#chat-history)

:::

This section will cover how to create conversational agents: chatbots that can interact with other systems and APIs using tools.

:::{.callout-note}

This how-to guide previously built a chatbot using [RunnableWithMessageHistory](https://python.langchain.com/api_reference/core/runnables/langchain_core.runnables.history.RunnableWithMessageHistory.html). You can access this version of the tutorial in the [v0.2 docs](https://python.langchain.com/v0.2/docs/how_to/chatbots_tools/).

The LangGraph implementation offers a number of advantages over `RunnableWithMessageHistory`, including the ability to persist arbitrary components of an application's state (instead of only messages).


## Setup

For this guide, we'll be using a [tool calling agent](https://langchain-ai.github.io/langgraph/concepts/agentic_concepts/#tool-calling-agent) with a single tool for searching the web. The default will be powered by [Tavily](/docs/integrations/tools/tavily_search), but you can switch it out for any similar tool. The rest of this section will assume you're using Tavily.

You'll need to [sign up for an account](https://tavily.com/) on the Tavily website, and install the following packages:

In [1]:
%pip install --upgrade --quiet langchain-community langchain-openai tavily-python langgraph

import getpass
import os

if not os.environ.get("OPENAI_API_KEY"):
    os.environ["OPENAI_API_KEY"] = getpass.getpass("OpenAI API Key:")

if not os.environ.get("TAVILY_API_KEY"):
    os.environ["TAVILY_API_KEY"] = getpass.getpass("Tavily API Key:")

OpenAI API Key: ········
Tavily API Key: ········


You will also need your OpenAI key set as `OPENAI_API_KEY` and your Tavily API key set as `TAVILY_API_KEY`.

## Creating an agent

Our end goal is to create an agent that can respond conversationally to user questions while looking up information as needed.

First, let's initialize Tavily and an OpenAI chat model capable of tool calling:

In [2]:
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_openai import ChatOpenAI

tools = [TavilySearchResults(max_results=1)]

# Choose the LLM that will drive the agent
# Only certain models support this
model = ChatOpenAI(model="gpt-4o-mini", temperature=0)

To make our agent conversational, we can also specify a prompt. Here's an example:

In [3]:
prompt = "You are a helpful assistant. You may not need to use tools for every query - the user may just want to chat!"

Great! Now let's assemble our agent:

In [4]:
from langgraph.prebuilt import create_react_agent

agent = create_react_agent(model, tools, state_modifier=prompt)

## Running the agent

Now that we've set up our agent, let's try interacting with it! It can handle both trivial queries that require no lookup:

In [5]:
from langchain_core.messages import HumanMessage

agent.invoke({"messages": [HumanMessage(content="I'm Nemo!")]})

{'messages': [HumanMessage(content="I'm Nemo!", additional_kwargs={}, response_metadata={}, id='f83a7d30-c6f3-486e-86c9-f0f3aef7befb'),
  AIMessage(content='Hi Nemo! How can I assist you today?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 11, 'prompt_tokens': 107, 'total_tokens': 118, 'completion_tokens_details': {'reasoning_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_e9627b5346', 'finish_reason': 'stop', 'logprobs': None}, id='run-077e4555-3e57-4888-bfa3-8f0ea30ce1c7-0', usage_metadata={'input_tokens': 107, 'output_tokens': 11, 'total_tokens': 118})]}

Or, it can use of the passed search tool to get up to date information if needed:

In [6]:
agent.invoke(
    {
        "messages": [
            HumanMessage(
                content="What is the current conservation status of the Great Barrier Reef?"
            )
        ],
    }
)

{'messages': [HumanMessage(content='What is the current conservation status of the Great Barrier Reef?', additional_kwargs={}, response_metadata={}, id='ab700e75-2f78-4f94-b05b-3d19741086b2'),
  AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_IZZQtFtjG6G7ZXEFH5HFfQu9', 'function': {'arguments': '{"query":"current conservation status of the Great Barrier Reef 2023"}', 'name': 'tavily_search_results_json'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 28, 'prompt_tokens': 116, 'total_tokens': 144, 'completion_tokens_details': {'reasoning_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_e9627b5346', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-bb187cde-b910-4de6-9a97-416a9f9e1e34-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'current conservation status of the Great Barrier Reef 2023'}, 'id': 'call_IZZQtFtjG6G7ZXEFH5HFfQu9', 'type': 'tool_call'}], 

## Conversational responses

Because our prompt contains a placeholder for chat history messages, our agent can also take previous interactions into account and respond conversationally like a standard chatbot:

In [7]:
from langchain_core.messages import AIMessage, HumanMessage

agent.invoke(
    {
        "messages": [
            HumanMessage(content="I'm Nemo!"),
            AIMessage(content="Hello Nemo! How can I assist you today?"),
            HumanMessage(content="What is my name?"),
        ],
    }
)

{'messages': [HumanMessage(content="I'm Nemo!", additional_kwargs={}, response_metadata={}, id='d0c42862-d9cc-42cd-a160-5190cb706b68'),
  AIMessage(content='Hello Nemo! How can I assist you today?', additional_kwargs={}, response_metadata={}, id='e0afc136-7a3e-45e9-bad7-066aa5c71ee6'),
  HumanMessage(content='What is my name?', additional_kwargs={}, response_metadata={}, id='72f1f141-d552-4e1e-879e-c37d86327793'),
  AIMessage(content='Your name is Nemo!', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 6, 'prompt_tokens': 130, 'total_tokens': 136, 'completion_tokens_details': {'reasoning_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_1bb46167f9', 'finish_reason': 'stop', 'logprobs': None}, id='run-317478fd-5a78-4808-921c-79b35dd02f77-0', usage_metadata={'input_tokens': 130, 'output_tokens': 6, 'total_tokens': 136})]}

If preferred, you can also add memory to the LangGraph agent to manage the history of messages. Let's redeclare it this way:

In [9]:
from langgraph.checkpoint.memory import MemorySaver

# highlight-start
memory = MemorySaver()
agent = create_react_agent(model, tools, state_modifier=prompt, checkpointer=memory)
# highlight-end

In [10]:
agent.invoke(
    {"messages": [HumanMessage("I'm Nemo!")]},
    config={"configurable": {"thread_id": "1"}},
)

{'messages': [HumanMessage(content="I'm Nemo!", additional_kwargs={}, response_metadata={}, id='c57b2862-ba9a-4088-ad3f-6cafdf6ede26'),
  AIMessage(content='Hi Nemo! How can I assist you today?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 11, 'prompt_tokens': 107, 'total_tokens': 118, 'completion_tokens_details': {'reasoning_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_1bb46167f9', 'finish_reason': 'stop', 'logprobs': None}, id='run-f7776bbb-3664-4e4b-9fe3-103d5adac7ac-0', usage_metadata={'input_tokens': 107, 'output_tokens': 11, 'total_tokens': 118})]}

And then if we rerun our wrapped agent executor:

In [11]:
agent.invoke(
    {"messages": [HumanMessage("What is my name?")]},
    config={"configurable": {"thread_id": "1"}},
)

{'messages': [HumanMessage(content="I'm Nemo!", additional_kwargs={}, response_metadata={}, id='c57b2862-ba9a-4088-ad3f-6cafdf6ede26'),
  AIMessage(content='Hi Nemo! How can I assist you today?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 11, 'prompt_tokens': 107, 'total_tokens': 118, 'completion_tokens_details': {'reasoning_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_1bb46167f9', 'finish_reason': 'stop', 'logprobs': None}, id='run-f7776bbb-3664-4e4b-9fe3-103d5adac7ac-0', usage_metadata={'input_tokens': 107, 'output_tokens': 11, 'total_tokens': 118}),
  HumanMessage(content='What is my name?', additional_kwargs={}, response_metadata={}, id='50dccb57-a644-40ae-9ba2-cd12c30292ca'),
  AIMessage(content='Your name is Nemo!', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 6, 'prompt_tokens': 130, 'total_tokens': 136, 'completion_tokens_details': {'reasoning_tok

This [LangSmith trace](https://smith.langchain.com/public/1a9f712a-7918-4661-b3ff-d979bcc2af42/r) shows what's going on under the hood.

## Further reading

Other types agents can also support conversational responses too - for more, check out the [agents section](/docs/tutorials/agents).

For more on tool usage, you can also check out [this use case section](/docs/how_to#tools).